# **MCTD**
This notebook contains the source code for running a black-box optimization algorithm using the MCTD algorithm.  
The notebook was tested on the Google colab platform with GPU accelration [https://research.google.com/colaboratory/faq.html]  
The Python version at Google Colab is 3.7.13 at the time of test.  

Additionally, it is possible to run the notebook locally by installing the required packages in 'requirement.txt'




## This section contains the package installation for Google Colab
If the notebook is running in a local environment with the necessary packages, this section may be *skipped*


### Install Pytorch, GPytorch, and scipy on Google Colab

In [1]:
# !pip3 install torch torchvision torchaudio
# !pip install gpytorch
# !pip install --upgrade scipy

# Importing packages

In [3]:
import sys
import os
import numbers
import copy
from copy import deepcopy
import warnings
import random
import math
from abc import abstractmethod
import json
import time
import pickle
import numpy as np

In [6]:
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=FutureWarning)
warnings.filterwarnings(action='ignore', category=Warning)
from sklearn import preprocessing
from scipy.stats import qmc 

import torch
print('Is cuda available: %s ' % torch.cuda.is_available())
from torch.quasirandom import SobolEngine

import gpytorch
from gpytorch.constraints.constraints import Interval
from gpytorch.distributions import MultivariateNormal
from gpytorch.kernels import MaternKernel, ScaleKernel
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.means import ConstantMean
from gpytorch.mlls import ExactMarginalLogLikelihood
from gpytorch.models import ExactGP

Is cuda available: True 


# Model Definitions

## Customized TuRBO model

The original *TuRBO* model source code is from https://github.com/uber-research/TuRBO  
Here, we made modifications to the source code to fit our algorithm by May.25.2022

The license of original *TuRBO* is as follows [https://github.com/uber-research/TuRBO/blob/master/LICENSE.md]:




"License" shall mean the terms and conditions for use, reproduction, and distribution as defined by the text below.
 
"You" (or "Your") shall mean an individual or Legal Entity exercising permissions granted by this License.
 
"Legal Entity" shall mean the union of the acting entity and all other entities that control, are controlled by, or are under common control with that entity. For the purposes of this definition, "control" means (i) the power, direct or indirect, to cause the direction or management of such entity, whether by contract or otherwise, or (ii) ownership of fifty percent (50%) or more of the outstanding shares, or (iii) beneficial ownership of such entity.
 
"Source" form shall mean the preferred form for making modifications, including but not limited to software source code, documentation source, and configuration files.
 
"Object" form shall mean any form resulting from mechanical transformation or translation of a Source form, including but not limited to compiled object code, generated documentation, and conversions to other media types.
 
"Work" shall mean the work of authorship, whether in Source or Object form, made available under this License.
 
This License governs use of the accompanying Work, and your use of the Work constitutes acceptance of this License.
 
You may use this Work for any non-commercial purpose, subject to the restrictions in this License. Some purposes which can be non-commercial are teaching, academic research, and personal experimentation. You may also distribute this Work with books or other teaching materials, or publish the Work on websites, that are intended to teach the use of the Work.
 
You may not use or distribute this Work, or any derivative works, outputs, or results from the Work, in any form for commercial purposes. Non-exhaustive examples of commercial purposes would be running business operations, licensing, leasing, or selling the Work, or distributing the Work for use with commercial products.
 
You may modify this Work and distribute the modified Work for non-commercial purposes, however, you may not grant rights to the Work or derivative works that are broader than or in conflict with those provided by this License. For example, you may not distribute modifications of the Work under terms that would permit commercial use, or under terms that purport to require the Work or derivative works to be sublicensed to others.

In return, we require that you agree:

1. Not to remove any copyright or other notices from the Work.
 
2. That if you distribute the Work in Source or Object form, you will include a verbatim copy of this License.
 
3. That if you distribute derivative works of the Work in Source form, you do so only under a license that includes all of the provisions of this License and is not in conflict with this License, and if you distribute derivative works of the Work solely in Object form you do so only under a license that complies with this License.
 
4. That if you have modified the Work or created derivative works from the Work, and distribute such modifications or derivative works, you will cause the modified files to carry prominent notices so that recipients know that they are not receiving the original Work. Such notices must state: (i) that you have changed the Work; and (ii) the date of any changes.
 
5. If you publicly use the Work or any output or result of the Work, you will provide a notice with such use that provides any person who uses, views, accesses, interacts with, or is otherwise exposed to the Work (i) with information of the nature of the Work, (ii) with a link to the Work, and (iii) a notice that the Work is available under this License.
 
6. THAT THE WORK COMES "AS IS", WITH NO WARRANTIES. THIS MEANS NO EXPRESS, IMPLIED OR STATUTORY WARRANTY, INCLUDING WITHOUT LIMITATION, WARRANTIES OF MERCHANTABILITY OR FITNESS FOR A PARTICULAR PURPOSE OR ANY WARRANTY OF TITLE OR NON-INFRINGEMENT. ALSO, YOU MUST PASS THIS DISCLAIMER ON WHENEVER YOU DISTRIBUTE THE WORK OR DERIVATIVE WORKS.
 
7. THAT NEITHER UBER TECHNOLOGIES, INC. NOR ANY OF ITS AFFILIATES, SUPPLIERS, SUCCESSORS, NOR ASSIGNS WILL BE LIABLE FOR ANY DAMAGES RELATED TO THE WORK OR THIS LICENSE, INCLUDING DIRECT, INDIRECT, SPECIAL, CONSEQUENTIAL OR INCIDENTAL DAMAGES, TO THE MAXIMUM EXTENT THE LAW PERMITS, NO MATTER WHAT LEGAL THEORY IT IS BASED ON. ALSO, YOU MUST PASS THIS LIMITATION OF LIABILITY ON WHENEVER YOU DISTRIBUTE THE WORK OR DERIVATIVE WORKS.
 
8. That if you sue anyone over patents that you think may apply to the Work or anyone's use of the Work, your license to the Work ends automatically.
 
9. That your rights under the License end automatically if you breach it in any way.
 
10. Uber Technologies, Inc. reserves all rights not expressly granted to you in this License.



In [7]:
def to_unit_cube(x, lb, ub):
    """Project to [0, 1]^d from hypercube with bounds lb and ub"""
    assert np.all(lb < ub) and lb.ndim == 1 and ub.ndim == 1 and x.ndim == 2
    xx = (x - lb) / (ub - lb)
    return xx


def from_unit_cube(x, lb, ub):
    """Project from [0, 1]^d to hypercube with bounds lb and ub"""
    assert np.all(lb < ub) and lb.ndim == 1 and ub.ndim == 1 and x.ndim == 2
    xx = x * (ub - lb) + lb
    return xx


def latin_hypercube(n_pts, dim):
    """Basic Latin hypercube implementation with center perturbation."""
    X = np.zeros((n_pts, dim))
    centers = (1.0 + 2.0 * np.arange(0.0, n_pts)) / float(2 * n_pts)
    for i in range(dim):  # Shuffle the center locataions for each dimension.
        X[:, i] = centers[np.random.permutation(n_pts)]

    # Add some perturbations within each box
    pert = np.random.uniform(-1.0, 1.0, (n_pts, dim)) / float(2 * n_pts)
    X += pert
    return X


class GP(ExactGP):
    def __init__(self, train_x, train_y, likelihood, lengthscale_constraint, outputscale_constraint, ard_dims):
        super(GP, self).__init__(train_x, train_y, likelihood)
        self.ard_dims = ard_dims
        self.mean_module = ConstantMean()
        base_kernel = MaternKernel(lengthscale_constraint=lengthscale_constraint, ard_num_dims=ard_dims, nu=2.5)
        self.covar_module = ScaleKernel(base_kernel, outputscale_constraint=outputscale_constraint)

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return MultivariateNormal(mean_x, covar_x)


def train_gp(train_x, train_y, use_ard, num_steps, hypers={}):
    """Fit a GP model where train_x is in [0, 1]^d and train_y is standardized."""
    assert train_x.ndim == 2
    assert train_y.ndim == 1
    assert train_x.shape[0] == train_y.shape[0]

    # Create hyper parameter bounds
    noise_constraint = Interval(5e-4, 0.2)
    if use_ard:
        lengthscale_constraint = Interval(0.005, 2.0)
    else:
        lengthscale_constraint = Interval(0.005, math.sqrt(train_x.shape[1]))  # [0.005, sqrt(dim)]
    outputscale_constraint = Interval(0.05, 20.0)

    # Create models
    with gpytorch.settings.cholesky_jitter(float=1e-1):
        likelihood = GaussianLikelihood(noise_constraint=noise_constraint).to(device=train_x.device, dtype=train_y.dtype)
        ard_dims = train_x.shape[1] if use_ard else None
        model = GP(
            train_x=train_x,
            train_y=train_y,
            likelihood=likelihood,
            lengthscale_constraint=lengthscale_constraint,
            outputscale_constraint=outputscale_constraint,
            ard_dims=ard_dims,
        ).to(device=train_x.device, dtype=train_x.dtype)

        # Find optimal model hyperparameters
        model.train()
        likelihood.train()

        # "Loss" for GPs - the marginal log likelihood
        mll = ExactMarginalLogLikelihood(likelihood, model)

        # Initialize model hypers
        if hypers:
            model.load_state_dict(hypers)
        else:
            hypers = {}
            hypers["covar_module.outputscale"] = 1.0
            hypers["covar_module.base_kernel.lengthscale"] = 0.5
            hypers["likelihood.noise"] = 0.005
            model.initialize(**hypers)

        # Use the adam optimizer
        optimizer = torch.optim.Adam([{"params": model.parameters()}], lr=0.1)

        for _ in range(num_steps):
            optimizer.zero_grad()
            output = model(train_x)
            loss = -mll(output, train_y)
            loss.backward()
            optimizer.step()

        # Switch to eval mode
        model.eval()
        likelihood.eval()

    return model



class Turbo1:
    """The TuRBO-1 algorithm.

    Parameters
    ----------
    f : function handle
    lb : Lower variable bounds, numpy.array, shape (d,).
    ub : Upper variable bounds, numpy.array, shape (d,).
    n_init : Number of initial points (2*dim is recommended), int.
    max_evals : Total evaluation budget, int.
    batch_size : Number of points in each batch, int.
    verbose : If you want to print information about the optimization progress, bool.
    use_ard : If you want to use ARD for the GP kernel.
    max_cholesky_size : Largest number of training points where we use Cholesky, int
    n_training_steps : Number of training steps for learning the GP hypers, int
    min_cuda : We use float64 on the CPU if we have this or fewer datapoints
    device : Device to use for GP fitting ("cpu" or "cuda")
    dtype : Dtype to use for GP fitting ("float32" or "float64")

    Example usage:
        turbo1 = Turbo1(f=f, lb=lb, ub=ub, n_init=n_init, max_evals=max_evals)
        turbo1.optimize()  # Run optimization
        X, fX = turbo1.X, turbo1.fX  # Evaluated points
    """

    def __init__(
        self,
        f,
        lb,
        ub,
        n_init,
        max_evals,
        batch_size=1,
        verbose=True,
        use_ard=True,
        max_cholesky_size=2000,
        n_training_steps=50,
        min_cuda=256,
        device="cpu",
        dtype="float32",
    ):

        # Very basic input checks
        assert lb.ndim == 1 and ub.ndim == 1
        assert len(lb) == len(ub)
        assert np.all(ub > lb)
        assert max_evals > 0 and isinstance(max_evals, int)
        assert n_init > 0 and isinstance(n_init, int)
        assert batch_size > 0 and isinstance(batch_size, int)
        assert isinstance(verbose, bool) and isinstance(use_ard, bool)
        assert max_cholesky_size >= 0 and isinstance(batch_size, int)
        assert n_training_steps >= 30 and isinstance(n_training_steps, int)
        assert max_evals > n_init and max_evals > batch_size
        assert device == "cpu" or device == "cuda"
        assert dtype == "float32" or dtype == "float64"
        if device == "cuda":
            assert torch.cuda.is_available(), "can't use cuda if it's not available"

        # Save function information
        self.f = f
        self.dim = len(lb)
        self.lb = lb
        self.ub = ub

        # Settings
        self.n_init = n_init
        self.max_evals = max_evals
        self.batch_size = batch_size
        self.verbose = verbose
        self.use_ard = use_ard
        self.max_cholesky_size = max_cholesky_size
        self.n_training_steps = n_training_steps

        # Hyperparameters
        self.mean = np.zeros((0, 1))
        self.signal_var = np.zeros((0, 1))
        self.noise_var = np.zeros((0, 1))
        self.lengthscales = np.zeros((0, self.dim)) if self.use_ard else np.zeros((0, 1))

        # Tolerances and counters
        self.n_cand = min(100 * self.dim, 5000)
        self.failtol = np.ceil(np.max([4.0 / batch_size, self.dim / batch_size]))
        self.succtol = 3
        self.n_evals = 0

        # Trust region sizes
        self.length_min = 0.5 ** 10
        self.length_max = 1.6
        self.length_init = 0.8

        # Save the full history
        self.X = np.zeros((0, self.dim))
        self.fX = np.zeros((0, 1))

        # Device and dtype for GPyTorch
        self.min_cuda = min_cuda
        self.dtype = torch.float32 if dtype == "float32" else torch.float64
        self.device = torch.device("cuda") if device == "cuda" else torch.device("cpu")
        if self.verbose:
            print("Using dtype = %s \nUsing device = %s" % (self.dtype, self.device))
            sys.stdout.flush()

        self.failcount = None
        self.succcount = None
        
        # Initialize parameters
        self._restart()



    def _restart(self):
        self._X = []
        self._fX = []
        if self.failcount is None:
            self.failcount = 0
        if self.succcount is None:
            self.succcount = 0
        self.length = self.length_init

    def _adjust_length(self, fX_next):
        if np.min(fX_next) < np.min(self._fX) - 1e-3 * math.fabs(np.min(self._fX)):
            self.succcount += 1
            self.failcount = 0
        else:
            self.succcount = 0
            self.failcount += 1

        if self.succcount == self.succtol:  # Expand trust region
            self.length = min([2.0 * self.length, self.length_max])
            self.succcount = 0
        elif self.failcount == self.failtol:  # Shrink trust region
            self.length /= 2.0
            self.failcount = 0

    def _create_candidates(self, X, fX, length, n_training_steps, hypers):
        """Generate candidates assuming X has been scaled to [0,1]^d."""
        # Pick the center as the point with the smallest function values
        # NOTE: This may not be robust to noise, in which case the posterior mean of the GP can be used instead
        assert X.min() >= 0.0 and X.max() <= 1.0

        # Standardize function values.
        mu, sigma = np.median(fX), fX.std()
        sigma = 1.0 if sigma < 1e-6 else sigma
        fX = (deepcopy(fX) - mu) / sigma

        # Figure out what device we are running on
        if len(X) < self.min_cuda:
            device, dtype = torch.device("cpu"), torch.float64
        else:
            device, dtype = self.device, self.dtype

        # We use CG + Lanczos for training if we have enough data
        with gpytorch.settings.max_cholesky_size(self.max_cholesky_size) and gpytorch.settings.cholesky_jitter(float=1e-1):
            X_torch = torch.tensor(X).to(device=device, dtype=dtype)
            y_torch = torch.tensor(fX).to(device=device, dtype=dtype)
            gp = train_gp(
                train_x=X_torch, train_y=y_torch, use_ard=self.use_ard, num_steps=n_training_steps, hypers=hypers
            )

            # Save state dict
            hypers = gp.state_dict()

        # Create the trust region boundaries
        x_center = X[fX.argmin().item(), :][None, :]
        weights = gp.covar_module.base_kernel.lengthscale.cpu().detach().numpy().ravel()
        weights = weights / weights.mean()  # This will make the next line more stable
        weights = weights / np.prod(np.power(weights, 1.0 / len(weights)))  # We now have weights.prod() = 1
        lb = np.clip(x_center - weights * length / 2.0, 0.0, 1.0)
        ub = np.clip(x_center + weights * length / 2.0, 0.0, 1.0)

        # Draw a Sobolev sequence in [lb, ub]
        seed = np.random.randint(int(1e6))
        sobol = SobolEngine(self.dim, scramble=True, seed=seed)
        pert = sobol.draw(self.n_cand).to(dtype=dtype, device=device).cpu().detach().numpy()
        pert = lb + (ub - lb) * pert

        # Create a perturbation mask
        prob_perturb = min(20.0 / self.dim, 1.0)
        mask = np.random.rand(self.n_cand, self.dim) <= prob_perturb
        ind = np.where(np.sum(mask, axis=1) == 0)[0]
        mask[ind, np.random.randint(0, self.dim - 1, size=len(ind))] = 1

        # Create candidate points
        X_cand = x_center.copy() * np.ones((self.n_cand, self.dim))
        X_cand[mask] = pert[mask]

        # Figure out what device we are running on
        if len(X_cand) < self.min_cuda:
            device, dtype = torch.device("cpu"), torch.float64
        else:
            device, dtype = self.device, self.dtype

        # We may have to move the GP to a new device
        gp = gp.to(dtype=dtype, device=device)

        # We use Lanczos for sampling if we have enough data
        with torch.no_grad(), gpytorch.settings.max_cholesky_size(self.max_cholesky_size):
            X_cand_torch = torch.tensor(X_cand).to(device=device, dtype=dtype)
            y_cand = gp.likelihood(gp(X_cand_torch)).sample(torch.Size([self.batch_size])).t().cpu().detach().numpy()

        # Remove the torch variables
        del X_torch, y_torch, X_cand_torch, gp

        # De-standardize the sampled values
        y_cand = mu + sigma * y_cand

        return X_cand, y_cand, hypers

    def _select_candidates(self, X_cand, y_cand):
        """Select candidates."""
        X_next = np.ones((self.batch_size, self.dim))
        for i in range(self.batch_size):
            # Pick the best point and make sure we never pick it again
            indbest = np.argmin(y_cand[:, i])
            X_next[i, :] = deepcopy(X_cand[indbest, :])
            y_cand[indbest, :] = np.inf
        return X_next

    def optimize(self, X_init, fX_init):
        """Run the full optimization process."""
        # Initialize parameters
        self._restart()
        fX_init = fX_init.reshape(-1, 1)
        
        
        # Update budget and set as initial data for this TR
        # self.n_evals += self.n_init
        ### reset how many iterations can run YZ
        self.n_init = len(fX_init)
        self.n_evals = self.n_init
        self._X = deepcopy(X_init)
        self._fX = deepcopy(fX_init)

        n_evals, fbest = self.n_evals, self._fX.min()
        
        # Append data to the global history
        self.X = np.vstack((self.X, deepcopy(X_init)))
        self.fX = np.vstack((self.fX, deepcopy(fX_init)))

        if self.verbose:
            fbest = self._fX.min()
            print(f"Starting from fbest = {fbest:.4}")
            sys.stdout.flush()

        # Thompson sample to get next suggestions
        while self.n_evals < self.max_evals and self.length >= self.length_min:
            # Warp inputs
            X = to_unit_cube(deepcopy(self._X), self.lb, self.ub)

            # Standardize values
            fX = deepcopy(self._fX).ravel()

            # Create th next batch
            X_cand, y_cand, _ = self._create_candidates(
                X, fX, length=self.length, n_training_steps=self.n_training_steps, hypers={}
            )
            X_next = self._select_candidates(X_cand, y_cand)

            # Undo the warping
            X_next = from_unit_cube(X_next, self.lb, self.ub)

            # Evaluate batch
            fX_next = np.array([[self.f(x)] for x in X_next])

            # Update trust region
            self._adjust_length(fX_next)

            # Update budget and append data
            self.n_evals += self.batch_size
            self._X = np.vstack((self._X, X_next))
            self._fX = np.vstack((self._fX, fX_next))

            if self.verbose and fX_next.min() < self.fX.min():
                n_evals, fbest = self.n_evals, fX_next.min()
                print(f"{n_evals}) New best: {fbest:.4}")
                sys.stdout.flush()

            # Append data to the global history
            self.X = np.vstack((self.X, deepcopy(X_next)))
            self.fX = np.vstack((self.fX, deepcopy(fX_next)))

                
                
    def run_steps(self, step=1):
        """Run the full optimization process."""
        while self.n_evals < self.max_evals:
            if len(self._fX) > 0 and self.verbose:
                n_evals, fbest = self.n_evals, self._fX.min()
                print(f"{n_evals}) Restarting with fbest = {fbest:.4}")
                sys.stdout.flush()

            # Initialize parameters
            self._restart()

            # Generate and evalute initial design points
            X_init = latin_hypercube(self.n_init, self.dim)
            X_init = from_unit_cube(X_init, self.lb, self.ub)
            fX_init = np.array([[self.f(x)] for x in X_init])

            # Update budget and set as initial data for this TR
            self.n_evals += self.n_init
            self._X = deepcopy(X_init)
            self._fX = deepcopy(fX_init)

            # Append data to the global history
            self.X = np.vstack((self.X, deepcopy(X_init)))
            self.fX = np.vstack((self.fX, deepcopy(fX_init)))

            if self.verbose:
                fbest = self._fX.min()
                print(f"Starting from fbest = {fbest:.4}")
                sys.stdout.flush()

            # Thompson sample to get next suggestions
            while self.n_evals < self.max_evals and self.length >= self.length_min:
                # Warp inputs
                X = to_unit_cube(deepcopy(self._X), self.lb, self.ub)

                # Standardize values
                fX = deepcopy(self._fX).ravel()

                # Create th next batch
                X_cand, y_cand, _ = self._create_candidates(
                    X, fX, length=self.length, n_training_steps=self.n_training_steps, hypers={}
                )
                X_next = self._select_candidates(X_cand, y_cand)

                # Undo the warping
                X_next = from_unit_cube(X_next, self.lb, self.ub)

                # Evaluate batch
                fX_next = np.array([[self.f(x)] for x in X_next])

                # Update trust region
                self._adjust_length(fX_next)

                # Update budget and append data
                self.n_evals += self.batch_size
                self._X = np.vstack((self._X, X_next))
                self._fX = np.vstack((self._fX, fX_next))

                if self.verbose and fX_next.min() < self.fX.min():
                    n_evals, fbest = self.n_evals, fX_next.min()
                    print(f"{n_evals}) New best: {fbest:.4}")
                    sys.stdout.flush()

                # Append data to the global history
                self.X = np.vstack((self.X, deepcopy(X_next)))
                self.fX = np.vstack((self.fX, deepcopy(fX_next)))

## Definition of nodes

In [8]:
class SearchNodeABS:
    """
    This class defines an abstracted node in any search model.
    
    obj = SearchNodeABS(identifier = None,
                X = None,
                y = None,
                parent = None,
                children = [],
                possible_moves = [],
                is_root = False,
                )
    Methods: 
        make_child(moves): create child nodes based on given moves. Must override.
    """
    
    
    def __init__(self,
                 X = None,
                 y = None,
                 **kwargs,
                ):
        self.X = X
        self.y = y

    def __call__(self):
        return self.X
    
    
    # the property section is reserved for overriding
    @property
    def X(self):
        return self._X
    
    @X.setter
    def X(self, val):
        self._X = val
    
    @property
    def y(self):
        return self._y

    @y.setter
    def y(self, val):
        self._y = val        

        
    def set_parameters(self, **kwargs):
        for key, val in kwargs.items():
            if getattr(self, key, 'Not Exist') != 'Not Exist':
                setattr(self, key, val)
        return

    
    
class RealVector(SearchNodeABS):
    """
    This class stores the pair of (X, y). 
    Further information may be stored in this class, but in current MCDesent we only use (X, y)
    """
    def __init__(self,       
                 X = None,
                 y = None,
                 lb = None,
                 ub = None,
                 comment = '',
                 **kwargs,
                ):

        # X is a 1D numpy vector in R^n
        # y is its value of a function at X     
        super().__init__(
                        X = X,
                        y = y,                   
                       )
        
        # lower/upper bound for the X value
        # can be array or a number
        self.lb = lb
        self.ub = ub
        self.comment = comment
        self.set_parameters(**kwargs)
        
        self.X = self._update_bounds(self.X)    
        return

    
    @property
    def X(self):
        return self._X

    
    @X.setter
    def X(self, val):
        if not isinstance(val, np.ndarray):
            if isinstance(val, numbers.Number): val = [val]
            try:
                val = np.array(val, dtype=float)
            except:
                raise ValueError('X value should be real number vector [R^n]')
        self._X = val
    

    
    def _update_bounds(self, array): 
        try:
            array = np.clip(array, self.lb, self.ub)
        except Exception as e:
            print("Fail to set boundary: "+str(e))  
        return array
    

        
class TreeNode:
    def __init__(self, 
                 stats,
                 fn,
                 identifier=None,
                 is_root=False,
                 children = None,
                 parent = None,
                 lb=None,
                 ub=None,                 
                 local_opt = 'stp',
                 uct_alpha = 0.0,
                 uct_explore = 1.0,
                 uct_improve = 1.0,
                 dims = None,
                 **kwargs,
                ):
        
        self.stats = stats   # current stats, typically a RealVector Node Object
        self.identifier = identifier
        self.is_root = is_root  
        self.children = children
        if self.children is None:
            self.children = []
        self.Ndescendats = 0
        self.update_parent(parent)

            
        if is_root: 
            self.level = 0
        else:
            try:
                self.level = self.parent.level + 1
            except:
                self.level = -1
        

        self.fn = fn
        self.lb = lb
        self.ub = ub
        if self.lb is None: self.lb = self.fn.lb
        if self.ub is None: self.ub = self.fn.ub    

        self.dims = dims
        if self.dims is None:
            try:
                self.dims = self.fn.dims
            except:
                pass


        self._uct = 0
        self.visit = 0
        self.local_opt = local_opt        
        self.uct_alpha = uct_alpha
        self.uct_explore = uct_explore
        self.uct_improve = uct_improve
        self.stats_pre = self.stats   # stats before last optimization run
        self.is_blocked = False
        
        
        # settings for use gpytorch to guess 
        self.min_gp = 20 # minimum number of samples to guess
        self.min_cuda = 256 # minimum number of samples to put to GPU
        self.max_cholesky_size = 4000 # max num of samples to use Cholesky, int
        self.gp_increment = 20
        self.device = torch.device("cuda")
        self.dtype = torch.float64
        self.curr_device = None
        self.curr_dtype = None
        self.use_ard = True   
        self.GP_trainstep=50
        self.GP_Nsample = -1
        self.uncert_ratio_limit = 2.0 # if the uncertainty of a guess is larger than the max uncertainty of saved sample times this ratio, then get ground truth
        self.saved_y_X = []
        self.saved_y_uncert = None 
        self.saved_marker = [] # some comments on the ground truth value
        self.gp = None
        self.turbo_length = None


        # normalization of data 
        self.X_scaler = preprocessing.MinMaxScaler(clip=True)
        self.y_scaler = preprocessing.StandardScaler()

        try:
            _lb = self.fn.lb
            _ub = self.fn.ub
            _t = np.vstack((_lb, _ub))
            self.X_scaler.fit(_t)
        except Exception as e:
            print('Failed at transforming Xscaler : %s'%(str(e)))
            pass


        self.stp_alpha_init = 1.0 # depreciated
        self.stp_alpha = self.stp_alpha_init
        self.stp_max_same_direction_check = 5
        self.stp_cumulate_improv = []
        self.stp_groundtruth_check = 0
        self.stp_consecutive_guess = 0
        self.max_stp_consecutive_guess = 20


        self.dy_recent = []  # recent improvements
        self.stats_0 = self.stats  # init stats
        self.y_0 = None 
        self.y_best = None


        if self.stats is not None:
            try:
                self.y_0 = self.cal_one(self.stats)
            except:
                pass

        self.y_best = self.y_0  # best found in this node and all children
        self.set_parameters(**kwargs)

        return



    def set_parameters(self, **kwargs):
        for key, val in kwargs.items():
            if getattr(self, key, 'Not Exist') != 'Not Exist':
                setattr(self, key, val)
        return    
    
    def update_parent(self, parent):
        self.parent = parent
        if (self.parent is not None) and (self not in self.parent.children):
            self.parent.children.append(self)


    def visit_once(self,):
        self.visit += 1
        
    def reset_visit(self,):
        self.visit = 0    
    
    def y_upward(self, y, dy=None):
        # dy is the improvement from computed leaf node
        # self.visit_once()
        if self.y_0 is None:
            self.y_0 = y

        if self.y_best is None: 
            self.y_best = y
            dy = 0.0
        else:
            if self.y_best > y:
                if dy is None: dy = self.y_best - y
                self.y_best = y
                print(' y_best improved for node %s' % self.identifier)
            else:
                if dy is None: dy = 0.0
                pass
        self.dy_recent.insert(0, dy)
        
        if self.parent is not None:
            self.parent.y_upward(self.y_best, dy)
        return


    def update_save(self, _y_X, update_parent=True, marker=None):
        try:
            self.saved_y_X = np.vstack((self.saved_y_X, _y_X))        
        except:
            self.saved_y_X = copy.deepcopy(_y_X)
            self.saved_y_X = self.saved_y_X.reshape(1, -1)
        self.saved_marker.append(marker)
        if update_parent and self.parent is not None:
            self.parent.update_save(_y_X, update_parent=update_parent, marker=marker)
        return


    def cal_one(self, _x):

        if isinstance(_x, SearchNodeABS):
            if _x.y is not None: 
                return _x.y # already calculated
            x = copy.deepcopy(_x.X)
        else:
            x = copy.deepcopy(_x)
        
        assert isinstance(x, np.ndarray) or isinstance(x, numbers.Number)

        y = self.get_groundtruth(x, marker='initial_node')

        if isinstance(y, np.ndarray) : 
            if y.dims == 1: y = y.item()
        if isinstance(_x, SearchNodeABS): 
            _x.y = y
            
        return y

    def get_groundtruth(self, x, marker='Turbo', ifAddCumulate=False):
        # Get the ground truth and add to self.saved_y_X
        _X = None # deepcopy X to save in this obj 
        _y = None # deepcopy y to save in this obj
        
        if isinstance(x, SearchNodeABS):
            _X = copy.deepcopy(x.X)
        elif isinstance(x, np.ndarray):
            _X = copy.deepcopy(x)
        
        if _X.ndim == 2 and len(_X) == 1:
            _X  = _X.reshape(-1)


        _X = np.clip(_X, self.fn.lb, self.fn.ub)
        _y = self.fn(_X)
        
#         if isinstance(_y, numbers.Number):
#             _y = np.array([_y])
        # case for functions:
        # case 1: x = [1 x n] , y = [1]        
        # case 2: x = [1 x n] , y = [n]        
        # case 3: x = [m x n] , y = [m]
        if _X.ndim == 1:  
            try:
                # case 1: x = [1 x n] , y = [1]
                if len(_X) != len(_y) : 
                    _X = _X.reshape(-1)
                    _y = _y.reshape(-1)
                else: # case 2: x = [1 x n] , y = [n] 
                    _X = _X.reshape(-1, 1)
                    _y = _y.reshape(-1, 1)
            except:
                pass
        else: 
            # case 3: x = [m x n] , y = [m]
            _y = _y.reshape(-1, 1)
            
        _y_X = np.hstack((_y, _X))

        self.update_save(_y_X, update_parent=True, marker=marker+"@"+self.identifier)            
        if isinstance(x, SearchNodeABS): x.y = copy.deepcopy(_y)
        
        
        try:
            if self.y_best is None:
                if self.dims is None:
                        self.dims = _X.shape[-1]
            elif _y < self.y_best:
                x2_vector = self.stats.__class__(_X, y=_y, parent=None, 
                                    lb=self.stats.lb, ub=self.stats.ub,
                                    )
                self.stats = x2_vector
                # self.y_best = _y # this update is included in y_upward
                if marker is not None:
                    print('    Find better value by : %s ; value = %.6f' %(marker,_y))

            if ifAddCumulate:
                stp_improv = self.y_best - _y if self.y_best > _y else 0.0
                self.stp_cumulate_improv.append(stp_improv)
        except:
            print('Fail at updating y_best at groundtruth, ', _y, self.y_best)

        self.y_upward(_y)
        return _y    



    def _choose_gp_node(self, node=None):
        if node is None:
            node = self
        while len(node.saved_y_X) < node.min_gp:
            if node.is_root:
                return node
            else:
                node = node.parent
        return node

    
    def _create_gp(self, _x=None):
        # Input: if use points close to _x for creating GP
        # Return: if successfully create a GP

        if len(self.saved_y_X) < self.min_gp: # if no enough samples, and is not a root node, do nothing
            return False
        
        if not self.is_root :
            device, dtype = torch.device("cpu"), torch.float64
        else:
            if len(self.saved_y_X) < self.min_cuda:
                device, dtype = torch.device("cpu"), torch.float64
            else:
                device, dtype = self.device, self.dtype  

        if self.gp is not None and self.GP_Nsample > 0:
            if len(self.saved_y_X) <= self.GP_Nsample + self.gp_increment:
                return True
            else:
                self._delete_gp()

        if _x is None:
            _use_sample = np.arange(len(self.saved_y_X))
        else:
            if _x.ndim == 1: _x = _x.reshape(1, -1)


            if len(self.saved_y_X) < self.min_cuda:
                _use_sample = np.arange(len(self.saved_y_X))
            else:
                _N = self.min_cuda

                _x_mean = np.mean(_x, axis=0)
                _dist = np.linalg.norm(self.saved_y_X[:, 1:]-_x_mean, axis=1)

                _use_sample = np.argsort(_dist)[:_N]     

        _X_scaled = self.X_scaler.transform(self.saved_y_X[_use_sample, 1: ])
        _y_scaled = self.y_scaler.fit_transform(self.saved_y_X[_use_sample, 0:1])
        
        with gpytorch.settings.max_cholesky_size(self.max_cholesky_size):
            X_torch = torch.tensor(_X_scaled).to(device=device, dtype=dtype)
            y_torch = torch.tensor(_y_scaled).to(device=device, dtype=dtype).reshape(-1)
            try:
                del self.gp
                self.gp = None
            except:
                pass
            self.gp = train_gp(
                train_x=X_torch, train_y=y_torch, 
                use_ard=self.use_ard, 
                num_steps=self.GP_trainstep, 
                hypers={},
            )
            
            self.saved_y_uncert = np.sqrt(self.gp(X_torch).variance.cpu().detach().numpy().reshape(-1))
            self.saved_y_uncert *= np.sqrt(self.y_scaler.var_)
            self.saved_y_uncert = self.saved_y_uncert.reshape(-1)
            del X_torch, y_torch

            self.curr_device = device
            self.curr_dtype = dtype
            self.GP_Nsample = len(self.saved_y_X)
        return True
    
    

    def _delete_gp(self):
        if self.gp is not None:
            del self.gp
            self.GP_Nsample = -1
            self.curr_device = None
            self.curr_dtype = None
            self.gp = None


    def guess(self, _x, isScaled=False):
        start = time.time()   
        isGPSuccess = self._create_gp()
        if not isGPSuccess:
            return  False, 0.0, 0., 0.

        device, dtype = self.curr_device, self.curr_dtype

        # scale the sample for prediction
        if _x.ndim == 1:
            _x = _x.reshape(1, -1)
        
        _x_scaled = _x
        if not isScaled:
            _x_scaled = self.X_scaler.transform(_x_scaled)
        _x_scaled = np.clip(_x_scaled, 0., 1.)  # XScaler uses MinMaxScale with [0, 1] as output


        _x_torch = torch.tensor(_x_scaled).to(device=device, dtype=dtype)
        
        pred_y_scaled = self.gp(_x_torch).mean.cpu().detach().numpy().reshape(-1, 1)        
        uncert_y_scaled = np.sqrt(self.gp(_x_torch).variance.cpu().detach().numpy().reshape(-1)) # use std as uncertainty        
        
        pred_y = self.y_scaler.inverse_transform(pred_y_scaled).reshape(-1)  # inverse transform
        uncert_y = uncert_y_scaled * np.sqrt(self.y_scaler.var_).reshape(-1)  # scale back the std 
        
        del _x_torch
        
        is_reliable = np.empty_like(uncert_y, dtype=int)
        is_reliable[:] = False
        is_reliable[uncert_y <= np.max(self.saved_y_uncert) * self.uncert_ratio_limit ] = True

        return True, pred_y, is_reliable, uncert_y   


#### Descent Optimization
    def opt_node(self, max_budget=1, opt=None, params={}, **kwargs):
        if opt == 'stp':
            method = self._local_stp
            self.stp_groundtruth_check = 0
            while self.stp_groundtruth_check < max_budget:
                method(**kwargs)
        elif opt == 'zigzag':  # zigzag is another version of STP with more guesses at local
            method = self._local_stp_zigzag
            self.stp_groundtruth_check = 0
            while self.stp_groundtruth_check < max_budget:
                method(**kwargs)
        return    


    
    def _local_stp_zigzag(self, 
                          iteration = 1,
                          cumulate_improve_thres = 5e-2,
                          cumulate_improve_count = 10,
                          use_alpha = True,
                          use_mask = False,
                          ):
        stats = self.stats
        _x_init = copy.deepcopy(stats.X)

        # make x as scaled
        _x_init = self.X_scaler.transform(_x_init.reshape(1,-1))

        n = self.visit if self.visit > 0 else 1
    
        Ncand = min(_x_init.shape[-1]*500, 100000)

        dx_cand_all = self._dx_sample(_x_init, 
                                      epoch=Ncand,
                                      range = self.stp_alpha,
                                      use_alpha = use_alpha,
                                      use_mask = use_mask,
                                     )

        node = self._choose_gp_node()

        xscale_best_curr = self.X_scaler.transform(self.stats.X.reshape(1,-1))
        y_best_curr = copy.deepcopy(self.stats.y)

        isSuccGuess, y_cand_all, isreliable_cand, y_cand_uncert_all = node.guess(_x_init + dx_cand_all, isScaled=True)
        if isSuccGuess:
            idx_cand = np.argmin(y_cand_all)
        else:
            idx_cand = np.random.randint(Ncand)
        dx_cand = dx_cand_all[idx_cand]

        x = _x_init
        _scale = 1.0
        _it = 0

        # here defines how much dx * _scale to examin
        leftChecked, rightChecked = 0., 0.
        left, right = -1., 1.
        curr = 0.
        shrink_fail = 0

        
        self.stp_cumulate_improv = []
        for i in range(iteration):
            succOnce = False
            isSuccess_right = False
            isSuccess_right, xscale_best_curr, y_best_curr = self._local_stp_try(x, dx_cand, (right-curr) * _scale, xscale_best_curr, y_best_curr)
            if isSuccess_right:
                succOnce = True
                new_t = right
                if new_t >= rightChecked:
                    right = new_t * 2.
                    rightChecked = right
                else:
                    right = (rightChecked + new_t)/2

                leftChecked = curr
                left = (curr + new_t)/2
                curr = new_t
                

            isSuccess_left = False
            if not isSuccess_right:
                isSuccess_left, xscale_best_curr, y_best_curr = self._local_stp_try(x, dx_cand, (left-curr) * _scale, xscale_best_curr, y_best_curr)
            if isSuccess_left:
                succOnce = True
                new_t = left
                if new_t <= leftChecked:
                    left = new_t * 2.
                    leftChecked = left
                else:
                    left = (leftChecked + new_t)/2
                rightChecked = curr # the closest checked position to the right
                right = (curr + new_t)/2 # next check of right
                curr = new_t
                

            if not isSuccess_right and not isSuccess_left:
                shrink_fail += 1
                if shrink_fail == 10:
                    break

                leftChecked = left
                left = (left + curr)/2
                
                rightChecked = right
                right = (right + curr)/2

            # exhausted stp search : break 
            if len(self.stp_cumulate_improv) > cumulate_improve_count:
                if sum(self.stp_cumulate_improv[-cumulate_improve_count:]) < abs(self.y_best) * cumulate_improve_thres:
                    break

        if succOnce :
            x_best_curr = self.X_scaler.inverse_transform(xscale_best_curr.reshape(1,-1)).reshape(-1)
            if x_best_curr not in self.saved_y_X[:, 1:]:
                self.get_groundtruth(x, marker = 'local_stp_zigzag_best_guess')
                self.stp_groundtruth_check += 1
        return




    def _local_stp(self, 
                   use_alpha = True,
                   use_mask = False,
                   **kwargs):
        _x_init = copy.deepcopy(self.stats.X)
        
        # make x as scaled
        _x_init = self.X_scaler.transform(_x_init.reshape(1, -1))
        n = self.visit if self.visit > 0 else 1
    
        Ncand = min(_x_init.shape[-1]*50, 100000)
        dx_cand_all = self._dx_sample(_x_init, 
                                     epoch=Ncand,
                                     range = self.stp_alpha,
                                     use_alpha = use_alpha,
                                     use_mask = use_mask,
                                     )



        node = self._choose_gp_node()

        xscale_best_curr = self.X_scaler.transform(self.stats.X.reshape(1,-1))
        y_best_curr = self.stats.y
        isSuccGuess, y_cand_all, isreliable_cand, y_cand_uncert_all = node.guess(_x_init + dx_cand_all, isScaled=True)
        if isSuccGuess:
            idx_cand = np.argmin(y_cand_all)
        else:
            idx_cand = np.random.randint(Ncand)
        
        dx_cand = dx_cand_all[idx_cand]

        x = _x_init

        
        # try with x + dx
        _scale = 1.0
        _it = 0

        isSuccess_posx = False
        isSuccess_negx = False
        isSuccess_posx_shrink = False
        isSuccess_negx_shrink = False
        isSuccess_posx, xscale_best_curr, y_best_curr = self._local_stp_try(x, dx_cand, _scale, xscale_best_curr, y_best_curr )
        if isSuccess_posx:  # if  success on +dx
            isSuccess_consecutive = True
            while isSuccess_consecutive:
                _scale *= 2
                isSuccess_consecutive, xscale_best_curr, y_best_curr  = self._local_stp_try(x, dx_cand, _scale, xscale_best_curr, y_best_curr )
        else:  # try with -dx
            isSuccess_negx, xscale_best_curr, y_best_curr  = self._local_stp_try(x, dx_cand, -1*_scale, xscale_best_curr, y_best_curr )
            isSuccess_consecutive = isSuccess_negx
            while isSuccess_consecutive:
                _scale *= 2
                isSuccess_consecutive, xscale_best_curr, y_best_curr  = self._local_stp_try(x, dx_cand, -1*_scale, xscale_best_curr, y_best_curr )


        if (not isSuccess_posx) and (not isSuccess_negx):
            _scale *= 0.5
            isSuccess_posx_shrink, xscale_best_curr, y_best_curr  = self._local_stp_try(x, dx_cand, _scale, xscale_best_curr, y_best_curr )
            isSuccess_consecutive = isSuccess_posx_shrink
            while isSuccess_consecutive:
                _scale *= 0.5
                isSuccess_consecutive, xscale_best_curr, y_best_curr  = self._local_stp_try(x, dx_cand, _scale, xscale_best_curr, y_best_curr )
            if not isSuccess_posx_shrink:
                isSuccess_negx_shrink, xscale_best_curr, y_best_curr  = self._local_stp_try(x, dx_cand, -1.*_scale, xscale_best_curr, y_best_curr )
                isSuccess_consecutive = isSuccess_negx_shrink
                while isSuccess_consecutive:
                    _scale *= 0.5
                    isSuccess_consecutive, xscale_best_curr, y_best_curr  = self._local_stp_try(x, dx_cand, -1.* _scale, xscale_best_curr, y_best_curr )


        succOnce = isSuccess_posx or isSuccess_negx or isSuccess_posx_shrink or isSuccess_negx_shrink
        if succOnce :
            x_best_curr = self.X_scaler.inverse_transform(xscale_best_curr.reshape(1, -1)).reshape(-1)
            if x_best_curr not in self.saved_y_X[:, 1:]:
                self.stp_groundtruth_check += 1
                self.get_groundtruth(x_best_curr, marker = 'local_stp_best_guess')
        return


    def _dx_sample(self, 
                      x = None,
                      epoch=10000,
                      range = 1.0,
                      use_alpha = False,
                      use_mask = False,
                      seed = None,
                    ):
        
        # return Latin Cubic sampling within [-1., +1.], 
        # can be scaled by GP CL

        if x is None:
            x = self.stats.X

        start = time.time()
        dims = x.shape[-1]

        sampler = 0
        if seed is None:
            sampler = qmc.LatinHypercube(d=dims)
        else:
            sampler = qmc.LatinHypercube(d=dims, seed=seed)

        dx_cand = 2 * sampler.random(n=epoch) - 1.0

        if use_alpha:
            node = self
            while len(node.saved_y_X) < node.min_gp and not node.is_root:
                node = node.parent
            try:
                node._create_gp()
                alpha = node.gp.covar_module.base_kernel.lengthscale.cpu().detach().numpy().ravel()
                
                # rescale to approximate the same length
                norm_old = np.linalg.norm(dx_cand, axis=1).reshape(-1, 1)
                dx_cand *= alpha 
                norm_new = np.linalg.norm(dx_cand, axis=1).reshape(-1, 1)
                dx_cand *= norm_old/norm_new

            except Exception as e:    
                print('failed at sampling with CL: ', str(e))
                pass
            
        # add a scale factor 
        dx_cand *= range

        return dx_cand


    def _local_stp_try(self, _x, dx, scale, _x_prev=None, _y_prev=None, ):

        # _x and dx are scaled by self.X_scaler
        start = time.time()
        if _x_prev is None:
            _x_prev = self.X_scaler.transform(self.stats.X.reshape(1, -1))
        if _y_prev is None:
            _y_prev = self.stats.y

        _x2 = np.clip(_x + scale * dx, 0., 1.)

        isSuccGuess = False
        if self.stp_consecutive_guess < self.max_stp_consecutive_guess:
            node = self._choose_gp_node()
            isSuccGuess, _y2, _, _ = node.guess(_x2, isScaled=True)
            if isSuccGuess:
                self.stp_consecutive_guess += 1
            else:
                self.stp_consecutive_guess = 0
                self.stp_groundtruth_check += 1
                _y2 = self.get_groundtruth(self.X_scaler.inverse_transform(_x2.reshape(1, -1)), marker = 'local_stp_try', ifAddCumulate=True,) 
        else:
            self.stp_consecutive_guess = 0
            self.stp_groundtruth_check += 1
            _y2 = self.get_groundtruth(self.X_scaler.inverse_transform(_x2.reshape(1, -1)), marker = 'local_stp_try', ifAddCumulate=True,)       # after this step, self.y_best is updated  
        is_success = False
        
        x_to_return = _x_prev
        y_to_return = _y_prev
        if _y2 < _y_prev:
            is_success = True
            x_to_return = _x2
            y_to_return = _y2
        end = time.time()
        return is_success, x_to_return, y_to_return



    @property
    def uct(self):
        self.cal_uct()
        return self._uct
    

    def cal_uct(self, pvisit=None, count_recent=40):
        if pvisit is None:
            if self.parent is not None:
                pvisit = self.parent.visit
            else:
                pvisit = 1
        assert isinstance(pvisit, int)
        
        if len(self.dy_recent) > 0:
            improv = np.sum(self.dy_recent[:count_recent])                
        else:
            improv = 0.

        qv = self.y_best 
        exploration = np.sqrt(np.log(np.max((pvisit, 1.0))) / np.max((self.visit, 1.0)) )

        self._uct = -qv \
                    + self.uct_improve * improv \
                    + self.uct_alpha * (self.y_0 - self.y_best) / np.max((self.visit, 1.0)) \
                    + self.uct_explore * exploration
        return    
    
    
    def create_kids_turbo(self, iteration=40, use_node = None, save_length=True):
        if use_node is None: use_node = self
        N_init = len(use_node.saved_y_X)
        
        y_best_prev = copy.deepcopy(self.y_best)
        with gpytorch.settings.cholesky_jitter(float=1e-1, double=1e-2):
            model = Turbo1(f=self.get_groundtruth, # Handle to objective function  
                        lb=self.lb,  # Numpy array specifying lower bounds
                        ub=self.ub,  # Numpy array specifying upper bounds
                        n_init=N_init,  # Number of initial bounds from an Latin hypercube design
                        max_evals =N_init + iteration,  # Maximum number of evaluations
                        batch_size=1,  # How large batch size TuRBO uses
                        verbose=False,  # Print information from each batch
                        use_ard=True,  # Set to true if you want to use ARD for the GP kernel
                        max_cholesky_size=2000,  # When we switch from Cholesky to Lanczos
                        n_training_steps=50,  # Number of steps of ADAM to learn the hypers
                        min_cuda=use_node.min_cuda,  # Run on the CPU for small datasets
                        device='cuda',  # "cpu" or "cuda"
                        dtype='float64',  # float64 or float32
                        )
            if save_length:
                if self.turbo_length is not None:
                    if self.turbo_length > model.length_min and self.turbo_length < model.length_max :
                        model.length_init = self.turbo_length
                        try:
                            model.succcount = self.turbo_succcount
                            model.failcount = self.turbo_failcount
                        except:
                            model.succcount = 0
                            model.failcount = 0
            model.optimize(use_node.saved_y_X[:, 1:], use_node.saved_y_X[:, 0])

            if save_length:
                self.turbo_length = copy.deepcopy(model.length)
                self.turbo_succcount = model.succcount
                self.turbo_failcount = model.failcount 
            self.stp_alpha = copy.deepcopy(model.length)

            del model
        
        return


## Definition of search

In [14]:
def MCDesentSearch( test_fn,
                    dims = None,
                    log = "./log.dat", 
                    max_steps = 100,            # Number of total iterations 
                    iter_descent = 10,          # Number of allowed function calls by descent in every iteration
                    iter_turbo = 20,            # Number of allowed function calls by TuRBO in every iteration
                    stp_internal_steps = 10,    # Number of allowed guess calls by descent 
                    width = 0.1,                # Relative distance scale at descent
                    stp_switch_thres = 35000.,  # When if STP wants to switch for more guess
                    rad_max_decay = 0.5,        # Max relative distance when create new node
                    rad_min_decay = 0.1,        # Min relative distance when create new node
                    node_uct_improve = 50.,     # C_d  for node
                    node_uct_explore = 1. ,     # C_p  for node
                    leaf_improve = 100.,        # C_d'' for leaf
                    leaf_explore = 10.,         # C_p'' for leaf
                    branch_explore = 1000.0,    # C_d' for branch 
                    **kwargs,
                  ):
        """
        This function defines the entire process of MCDesent search.
        """

        try:
            test_fn = fn(dims = dims)
        except:
            test_fn = fn()
            dims = test_fn.dims

        print("=========================================================")
        print("start evaluating function %s in dim %d "%(fn, dims))
        try:
            print(" lb = %f,  ub = %f"%(test_fn.lb[0], test_fn.ub[0]))
        except:
            print(" lb = %f,  ub = %f"%(test_fn.lb, test_fn.ub))

        torch.cuda.empty_cache()

        # control how many improvements are considered in the history
        explore_recent_improve_count = 3*(iter_descent + iter_turbo)  # np.sum(dy_recent[:explore_recent_improve_count])

        
        test_fn.counter = 0
        step = 0
        
        
        sample = np.random.rand(dims)
        val = test_fn.lb + (test_fn.ub - test_fn.lb) * sample
        vec_root = RealVector(val, 
                    lb=test_fn.lb, ub=test_fn.ub,
                    )
        root = TreeNode(vec_root, test_fn, is_root=True, 
                        identifier = 'root',
                        device = torch.device("cuda"),
                        min_gp = 64,
                        uct_explor = node_uct_explore,
                        uct_improv = node_uct_improve,
                        )

        
        for istep in range(max_steps):

            # Choose the correct path and leaf
            curr_node = root
            print('Step %d , curr queries: %d, curr_best: %f'%(istep, len(root.saved_y_X), root.y_best ))
            print('Finding path in step %d  ...'%istep)

            to_explore_branch = False

            while curr_node.children:
                to_explore_branch = False

                to_explore_node = lambda: None
                to_explore_node.identifier = curr_node.identifier + " @IfExplore"
                to_explore_node.y_best = curr_node.y_best
                to_explore_node.dy_recent = []
                to_explore_node.uct = 0

                qv = []
                for k in curr_node.children:
                    qv.append(k.y_best)
                qv = np.mean(qv)
                to_explore_node.uct += -qv

                pvisit = 1
                try:
                    pvisit = curr_node.parent.visit
                except:
                    pass
                to_explore_node.uct += branch_explore * np.sqrt(np.log(np.max((curr_node.visit, 1.0))) / len(curr_node.children) )  # the more children, the less exploration


                check_list = [to_explore_node, *curr_node.children]
                check_list.sort(key=lambda node: node.uct, reverse=True) 
                for kid in check_list:
                    print('         children y_best/uct/dy_recent:', kid.identifier, kid.y_best, kid.uct, kid.dy_recent[:10], )

                if check_list[0] == to_explore_node:
                    to_explore_branch = True
                    curr_node = curr_node
                else:
                    curr_node = check_list[0]
                    print("  Choose: " , curr_node.identifier, curr_node.y_best, curr_node.uct)


                
                if to_explore_branch:

                    print('      To explore branch node : %s ' % curr_node.identifier )
                    # create an exploration kid
                    sample = curr_node._dx_sample(epoch=1, use_alpha=False)[0]

                    # Use relative distance from boundaries
                    # + means towards ub, - means towards lb
                    # e.g., +0.2 means to move 20% to ub, and -0.15 means move 15% to lb
                    var_dist_p = (rad_max_decay**curr_node.level)     * sample               # random percentage towards boundaries
                    min_dist_p = (rad_min_decay**(curr_node.level+1)) * np.sign(var_dist_p)  # minimum percentage towards boundaries

                    dist = test_fn.ub - curr_node.stats.X
                    dist_neg = curr_node.stats.X - test_fn.lb
                    mask = var_dist_p<0
                    dist[mask] = dist_neg[mask]  # this is the 100% distance value from x to boundary

                    perc_mtx = np.ones_like(var_dist_p)
                    final_p =  (perc_mtx-np.abs(min_dist_p)) * var_dist_p + min_dist_p
                    var_dist = dist * final_p

                    val = curr_node.stats.X + var_dist
                    val = np.clip(val, test_fn.lb, test_fn.ub)
                    vector = RealVector(val,
                                        lb=test_fn.lb, ub=test_fn.ub,
                                        )
                    node = TreeNode(vector, test_fn, parent=curr_node, 
                                    identifier = curr_node.identifier + "_"  + "%03d"%(len(curr_node.children)),
                                    uct_improve = node_uct_improve,
                                    uct_explore = node_uct_explore,
                                    )
                    
                    var_norm = np.linalg.norm(var_dist, np.inf)
                    # min_norm = np.linalg.norm(min_dist, np.inf)
                    print('   var norm = %f '%(var_norm))
                    # after creating, set it as to opt
                    curr_node = node
                    break


            to_explore_leaf = False
            if not to_explore_branch: # if it is set to explore as a branch node, do not check again
                if curr_node.visit < 3:
                    # to few visits to this node, then always exploit
                    print('LEAF node min exploitation not reached:%s' % curr_node.identifier)
                    to_explore_leaf = False  
                else:
                    thres =  leaf_explore * np.sqrt(np.max((curr_node.visit, 1.0)))                  
                    Nimprov = explore_recent_improve_count
                    if curr_node.y_best == root.y_best:
                        Nimprov = int(3*Nimprov)
                    improve = np.sum(curr_node.dy_recent[:Nimprov]) 
                    improve_std = improve * leaf_improve
                    find_best = curr_node.y_best
                    find_best_std = -1.0 * find_best 
                    to_explore_leaf = (find_best_std + improve_std) <= thres
                    print(' To explore leaf? ... best_std = %f    improve_std = %f   thres = %f'%(find_best_std, improve_std, thres))

                    
            if to_explore_leaf:
                print('      To explore LEAF node : %s ' % curr_node.identifier )
                # create an inherit kid
                Ncopy = min(40, iter_descent + iter_turbo, len(curr_node.saved_y_X))
                _x_center = copy.deepcopy(curr_node.stats.X)
                _dist = np.linalg.norm(_x_center - curr_node.saved_y_X[:, 1:], axis=-1)
                _idx_Nclosest = np.argpartition(_dist, Ncopy)


                dup_kid = TreeNode(copy.deepcopy(curr_node.stats), test_fn, parent = curr_node,
                                    identifier = curr_node.identifier + "_000",
                                    saved_y_X = copy.deepcopy(curr_node.saved_y_X[_idx_Nclosest[:Ncopy]]),
                                    dy_recent = copy.deepcopy(curr_node.dy_recent[-40:]),
                                    visit = 1,
                                    turbo_length = curr_node.turbo_length,
                                    uct_improve = node_uct_improve,
                                    )


                # create an exploration kid
                sample = curr_node._dx_sample(epoch=1, use_alpha=False, use_mask=False)[0]

                # Use relative distance from boundaries
                # + means towards ub, - means towards lb
                # e.g., +0.2 means to move 20% to ub, and -0.15 means move 15% to lb
                var_dist_p = (rad_max_decay**curr_node.level)     * sample               # random percentage towards boundaries
                min_dist_p = (rad_min_decay**(curr_node.level+1)) * np.sign(var_dist_p)  # minimum percentage towards boundaries


                dist = test_fn.ub - curr_node.stats.X
                dist_neg = curr_node.stats.X - test_fn.lb
                mask = var_dist_p<0
                dist[mask] = dist_neg[mask]  # this is the 100% distance value from x to boundary

                perc_mtx = np.ones_like(var_dist_p)
                final_p =  (perc_mtx-np.abs(min_dist_p)) * var_dist_p + min_dist_p

                var_dist = dist * final_p

                val = curr_node.stats.X + var_dist
                val = np.clip(val, test_fn.lb, test_fn.ub)

                vector = RealVector(val,
                                    lb=test_fn.lb, ub=test_fn.ub,
                                    )
                node = TreeNode(vector, test_fn, parent=curr_node, 
                                identifier = curr_node.identifier + "_001" ,
                                turbo_length = curr_node.turbo_length,
                                uct_improve = node_uct_improve,
                                )
                var_norm = np.linalg.norm(var_dist, np.inf)
                curr_node = node




            # opt curr_node
            curr_node.visit_once()
            _n = curr_node.parent
            while _n:
                _n.visit_once()
                _n = _n.parent

            curr_node.stp_alpha = 1./(curr_node.level+1) * width / np.sqrt(curr_node.visit)
            if curr_node.y_best > stp_switch_thres :
                curr_node.opt_node(max_budget = iter_descent,
                            use_alpha= True,
                            use_mask = False,
                            )
            else:
                curr_node.opt_node(opt='zigzag',
                                max_budget = iter_descent,
                                use_alpha= True,
                                    use_mask = False,
                                    iteration = stp_internal_steps,
                                    cumulate_improve_count = 10,
                                    )
            if iter_turbo >0:
                curr_node.create_kids_turbo(iteration = iter_turbo)

            np.savetxt(log, root.saved_y_X, header='y X')

# Test functions definition

## Synthetic functions

In [10]:
class Ackley:
    def __init__(self, dims=10):
        self.dims      = dims
        self.lb        = -5 * np.ones(dims)
        self.ub        =  10 * np.ones(dims)
        self.counter   = 0
        
        
    def __call__(self, x):
        if isinstance(x, numbers.Number):
            x = np.array([x])
        if isinstance(x, list):
            x = np.array(x)            
        if not isinstance(x, np.ndarray):
            x = x.X          

        assert len(x) == self.dims
        assert x.ndim == 1
        x = np.clip(x, self.lb, self.ub)
        assert np.all(x <= self.ub) and np.all(x >= self.lb)
        
        self.counter += 1
        
        result = (-20*np.exp(-0.2 * np.sqrt(np.inner(x,x) / x.size )) -np.exp(np.cos(2*np.pi*x).sum() /x.size) + 20 +np.e )
        return result
    
    
class Michalewicz:
    def __init__(self, dims=2, m=10):
        self.dims      = dims
        self.lb        = np.zeros(self.dims)
        self.ub        = np.pi * np.ones(self.dims)
        self.m         = 10
        self.counter   = 0
        
        
    def __call__(self, x):
        if isinstance(x, numbers.Number):
            x = np.array([x])
        if isinstance(x, list):
            x = np.array(x)            
        if not isinstance(x, np.ndarray):
            x = x.X          

        assert len(x) == self.dims
        assert x.ndim == 1
        x = np.clip(x, self.lb, self.ub)
        try:
            assert np.all(x <= self.ub) and np.all(x >= self.lb)
        except:
            x = np.clip(x, self.lb, self.ub)
        self.counter += 1
               
            
        a_sin = np.sin(x)
        b = x**2  * (np.arange(x.shape[0]) + 1) / np.pi
        b_sin = np.sin(b)**(2*self.m)
        
        result = -np.sum(a_sin*b_sin)
        return result    


## NAS problem

In [11]:
class NB201_Cifar10:
    
    def __init__(self, file=None):
        self.dims         = 6
        self.lb           =  0.51 * np.ones(self.dims)
        self.ub           =  5.49 * np.ones(self.dims)
        self.counter      = 0
        self.file         = file
        if self.file is None:
            self.file = './Data/Data_nb201_cifar10.pickle'
        with open(self.file, 'rb') as fp:
            self.data = pickle.load(fp)
        
    
    def __call__(self, x):
        
        if isinstance(x, numbers.Number):
            x = np.array([x])     
        if isinstance(x, list):
            x = np.array(x)            
        if not isinstance(x, np.ndarray):
            x = x.X  
            
            
        self.counter += 1
        assert len(x) == self.dims
        assert x.ndim == 1
        x = np.clip(x, self.lb, self.ub)
        assert np.all(x <= self.ub) and np.all(x >= self.lb)
        
        
        x_key = tuple(np.rint(x).astype(int))                
        eval_loss = 100 - self.data[x_key]

        return eval_loss


class NB201_Cifar100:
    
    def __init__(self, file=None):
        self.dims         = 6
        self.lb           =  0.51 * np.ones(self.dims)
        self.ub           =  5.49 * np.ones(self.dims)
        self.counter      = 0
        self.file         = file
        if self.file is None:
            self.file = './Data/Data_nb201_cifar100.pickle'
            
        with open(self.file, 'rb') as fp:
            self.data = pickle.load(fp)
        
    
    def __call__(self, x):
        
        if isinstance(x, numbers.Number):
            x = np.array([x])     
        if isinstance(x, list):
            x = np.array(x)            
        if not isinstance(x, np.ndarray):
            x = x.X  
            
            
        self.counter += 1
        assert len(x) == self.dims
        assert x.ndim == 1
        x = np.clip(x, self.lb, self.ub)
        assert np.all(x <= self.ub) and np.all(x >= self.lb)
        
        
        x_key = tuple(np.rint(x).astype(int))                
        eval_loss = 100 - self.data[x_key]

        return eval_loss

# MCTD tests

In [ ]:
output_dir = './'

### Ackley 50

In [ ]:
for jj in range(5):
    fn = Ackley
    dims = 50

    log = os.path.join(output_dir, fn.__name__ + str(dims) + '%02d'%jj+'.dat')


    MCDesentSearch(fn, dims, log,
                    max_steps = 10,
                    iter_descent = 20,
                    iter_turbo = 20,
                    node_uct_improve = 10.,
                    node_uct_explore = 0.5,
                    leaf_improve = 50.,
                    leaf_explore = 0.1,
                    branch_explore = .1,
                    width = 0.2,
                    stp_switch_thres = 10.,
                  )

### Ackley 100

In [ ]:
for jj in range(5):
    fn = Ackley
    dims = 100

    log = os.path.join(output_dir, fn.__name__ + str(dims) + '%02d'%jj+'.dat')


    MCDesentSearch(fn, dims, log,
                    max_steps = 80,
                    iter_descent = 20,
                    iter_turbo = 20,
                    node_uct_improve = 20.,
                    node_uct_explore = 0.5,
                    leaf_improve = 5.,
                    leaf_explore = 0.1,
                    branch_explore = .1,
                    width = 0.2,
                    stp_switch_thres = 4.,
                  )


### Mich 100

In [ ]:
for jj in range(5):
    fn = Michalewicz
    dims = 100

    log = os.path.join(output_dir, fn.__name__ + '%02d'%jj+'.dat')


    MCDesentSearch(fn, dims, log,
                    max_steps = 100,
                    iter_descent = 10,
                    iter_turbo = 20,
                    node_uct_improve = 50.,
                    node_uct_explore = 1.,
                    leaf_improve = 1.,
                    leaf_explore = 10.,  
                    branch_explore = .2,
                    width = 0.02,
                    stp_switch_thres = -30.,
                  )

### NAB-Cifar-10

In [ ]:
for jj in range(5):
    fn = NB201_Cifar10
    log = os.path.join(output_dir, fn.__name__ + '%02d'%jj+'.dat')

    MCDesentSearch(fn, log,
                    max_steps = 100,
                    iter_descent = 5,
                    iter_turbo = 20,
                    node_uct_improve = 50.,
                    node_uct_explore = 1.,
                    leaf_improve = 100.,
                    leaf_explore = 10.,
                    branch_explore = 1.0,
                    stp_switch_thres = 5.,
                    width = 0.5,
                  )

### NAB-Cifar-100

In [ ]:
for jj in range(5):
    fn = NB201_Cifar100
    log = os.path.join(output_dir, fn.__name__ + '%02d'%jj+'.dat')

    MCDesentSearch(fn, log,
                    max_steps = 100,
                    iter_descent = 5,
                    iter_turbo = 20,
                    node_uct_improve = 50.,
                    node_uct_explore = 1.,
                    leaf_improve = 100.,
                    leaf_explore = 10.,
                    branch_explore = 1.0,
                    stp_switch_thres = 5.,
                    width = 0.5,
                  )